In [1]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array
from PIL import Image

In [ ]:
SIZE = 128  
NoofIMGs = 9501
batch_size = 16

path_1 = r"C:\Users\vuvie\Downloads\ship data\without_mask"
path_2 = r"C:\Users\vuvie\Downloads\ship data\with_mask"


def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

files_1 = sorted_alphanumeric([f for f in os.listdir(path_1) if os.path.isfile(os.path.join(path_1, f))])[NoofIMGs:]
files_2 = sorted_alphanumeric([f for f in os.listdir(path_2) if os.path.isfile(os.path.join(path_2, f))])[NoofIMGs:]
without_mask_paths = [os.path.join(path_1, fname) for fname in files_1]
with_mask_paths = [os.path.join(path_2, fname) for fname in files_2]


def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')  # Đọc ảnh và chuyển sang RGB
    img = img.resize((SIZE, SIZE))  # Resize ảnh
    img = np.array(img) / 127.5 - 1.0  # Chuẩn hóa về [-1, 1]
    return tf.convert_to_tensor(img, dtype=tf.float32)

def create_dataset(without_mask_paths, with_mask_paths):
    def generator():
        for without_mask_path, with_mask_path in zip(without_mask_paths, with_mask_paths):
            without_mask_img = load_and_preprocess_image(without_mask_path)
            with_mask_img = load_and_preprocess_image(with_mask_path)
            yield without_mask_img, with_mask_img

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(SIZE, SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(SIZE, SIZE, 3), dtype=tf.float32),
        )
    )
    return dataset

train_dataset = create_dataset(without_mask_paths, with_mask_paths)

train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)


for without_mask_img, with_mask_img in train_dataset.take(1):
    print("Without mask image shape:", without_mask_img.shape)
    print("With mask image shape:", with_mask_img.shape)


Without mask image shape: (16, 128, 128, 3)
With mask image shape: (16, 128, 128, 3)


In [7]:
loaded_generator = tf.keras.models.load_model(r"C:\Users\vuvie\Downloads\generator_final.h5")

In [8]:
def calculate_psnr(real_images, generated_images):
    psnr = tf.image.psnr(real_images, generated_images, max_val=1.0) 
    return psnr


In [9]:
def calculate_ssim(real_images, generated_images):
    ssim = tf.image.ssim(real_images, generated_images, max_val=1.0) 
    return ssim


In [10]:
psnr_scores = []
ssim_scores = []

for without_mask_img, with_mask_img in train_dataset: 
    generated_imgs = loaded_generator(without_mask_img, training=False)

    for real_img, fake_img in zip(with_mask_img, generated_imgs):
        psnr_value = calculate_psnr(real_img, fake_img)
        ssim_value = calculate_ssim(real_img, fake_img)

        psnr_scores.append(psnr_value.numpy())
        ssim_scores.append(ssim_value.numpy())

avg_psnr = np.mean(psnr_scores)
avg_ssim = np.mean(ssim_scores)

print(f"Average PSNR: {avg_psnr:.4f}")
print(f"Average SSIM: {avg_ssim:.4f}")


Average PSNR: 9.3640
Average SSIM: 0.4956
